<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Trades-Endpoints/blob/main/Place_order_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# POST /openApi/swap/v2/trade/order

# UID 速率限制：5 次/秒
# 按 IP 分組速率限制：

# API KEY 權限需求：永續合約交易

In [ ]:
# 匯入時間模組
import time
# 匯入requests模組，用來發送HTTP請求
import requests
# 匯入hmac模組，用於加密簽名
import hmac
# 匯入sha256雜湊算法
from hashlib import sha256

In [ ]:
# API 主機網址
APIURL = "https://open-api.bingx.com"
# API 金鑰，請填寫你的APIKEY
APIKEY = ""
# 金鑰密鑰，請填寫你的SECRETKEY
SECRETKEY = ""

In [ ]:
# 範例函式，發送下單請求
def demo():
    # 空的請求主體
    payload = {}
    # API路徑
    path = '/openApi/swap/v2/trade/order'
    # HTTP請求方法
    method = "POST"

    # 下單參數
    paramsMap = {
        "symbol": "BTC-USDT",  # 交易對
        "side": "BUY",         # 買進
        "positionSide": "LONG",# 多單
        "type": "MARKET",      # 市價單
        "quantity": 5,         # 下單數量
        "takeProfit": "{\"type\": \"TAKE_PROFIT_MARKET\", \"stopPrice\": 31968.0,\"price\": 31968.0,\"workingType\":\"MARK_PRICE\"}"
        # 止盈條件，需以字串傳遞
    }

    # 解析參數並加上timestamp
    paramsStr = parseParam(paramsMap)
    # 發送API請求
    return send_request(method, path, paramsStr, payload)

In [ ]:
# 計算簽名
def get_sign(api_secret, payload):
    # 使用 HMAC-SHA256 生成簽名
    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    # 輸出簽名
    print("sign=" + signature)
    # 回傳簽名
    return signature

In [ ]:
# 發送API請求
def send_request(method, path, urlpa, payload):
    # 組合請求網址，包含簽名
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    # 輸出請求網址
    print(url)

    # 設定請求標頭，放入API KEY
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    # 發送HTTP請求
    response = requests.request(method, url, headers=headers, data=payload)
    # 回傳伺服器回應內容
    return response.text

In [ ]:
# 參數字典轉換成排序字串，並加上timestamp
def parseParam(paramsMap):
    # 取得排序後的key
    sortedKeys = sorted(paramsMap)
    # 將參數組合成字串
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])
    # 如果不是空字串，加上&timestamp
    if paramsStr != "":
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 若為空，直接加timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
# 主程式區塊
if __name__ == '__main__':
    # 執行demo函式，印出結果
    print("demo:", demo())